In [1]:
from google.colab import files

In [ ]:
files.upload()

In [3]:
import json

In [4]:
with open('out.json', 'r') as f:
    tracks = json.load(f)

In [5]:
import tensorflow as tf

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay as pcd


import numpy as np
import random

In [7]:
tracks_train = tracks[0:int(0.8*len(tracks))]
tracks_valid = tracks[int(0.8*len(tracks)):]
len(tracks), len(tracks_train), len(tracks_valid)

(748, 598, 150)

In [8]:
  maxlen = 20
  predlen = 10
  step = 1

In [9]:
def build_dataset(tracks_list, maxlen, predlen, step):
  sequences = []
  next_points = []
  for track in tracks_list:
    for i in range(0, len(track)- maxlen-predlen, step):
        sequences.append(track[i : i + maxlen])
        next_points.append(track[i+maxlen: i+maxlen+predlen])
  x = np.zeros((len(sequences), maxlen, 2))
  y = np.zeros((len(sequences), predlen*2)) # the prediction shape is flattened to make the architecture simpler
  for i, seq in enumerate(sequences):
      for j, p in enumerate(seq):
          x[i, j, 0] = p[0][0]
          x[i, j, 1] = p[0][1]
      temp = np.array([[p[0][0], p[0][1]] for p in next_points[i]]).flatten()
      y[i] = temp
  return x, y

In [10]:
x, y = build_dataset(tracks, maxlen, predlen, step)

In [11]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(maxlen, 2)),
        layers.LSTM(64),
        layers.Dense(128, activation='relu'),
        layers.Dense(32, activation='relu'),
        # layers.Dropout(0.2),
        layers.Dense(2*predlen, activation=None), # Linear activation
    ]
)

In [12]:
val_split = 0.2
batch_size = 32

In [13]:
steps = int((len(x)/batch_size) * (1-val_split))
boundaries = [a * steps for a in [10, 50, 155]]
values = [0.01, 0.001, 0.0001, 0.00005]
sch = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

optimizer = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss="mae", optimizer=optimizer)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                17152     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
Total params: 30,260
Trainable params: 30,260
Non-trainable params: 0
_________________________________________________________________


In [15]:
checkpoint_filepath = 'checkpoint55.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [16]:
epochs = 200
model.fit(x, y, batch_size=batch_size,
          validation_split=val_split, 
          epochs=epochs, shuffle=True, callbacks=model_checkpoint_callback,
          verbose = 1)

Epoch 1/200
357/357 [==============================] - 9s 5ms/step - loss: 223.9358 - val_loss: 248.7549
Epoch 2/200
357/357 [==============================] - 1s 4ms/step - loss: 183.8480 - val_loss: 201.5822
Epoch 3/200
357/357 [==============================] - 1s 4ms/step - loss: 153.5984 - val_loss: 195.4159
Epoch 4/200
357/357 [==============================] - 1s 4ms/step - loss: 147.8205 - val_loss: 191.8136
Epoch 5/200
357/357 [==============================] - 1s 4ms/step - loss: 144.7894 - val_loss: 189.1308
Epoch 6/200
357/357 [==============================] - 1s 4ms/step - loss: 141.0346 - val_loss: 182.0960
Epoch 7/200
357/357 [==============================] - 1s 4ms/step - loss: 130.4404 - val_loss: 162.1021
Epoch 8/200
357/357 [==============================] - 1s 4ms/step - loss: 106.3393 - val_loss: 117.0367
Epoch 9/200
357/357 [==============================] - 2s 4ms/step - loss: 60.5986 - val_loss: 38.2078
Epoch 10/200
357/357 [==============================] - 1

In [ ]:
 optimizer = keras.optimizers.Adam(learning_rate=0.000001)
model.compile(loss="mae", optimizer=optimizer)
epochs = 30
model.fit(x, y, batch_size=batch_size,
          validation_split=val_split, 
          epochs=epochs, shuffle=True, verbose = 1)

In [17]:
model.save('trained_colab2_55.h5')

In [ ]:
model.load_weights('trained_colab2_83.h5')

In [18]:
files.download('/content/trained_colab2_55.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls

out.json  sample_data  trained_colab2.h5


In [ ]:
predictions = model(x[0:100])

In [ ]:
predictions.shape

TensorShape([100, 20])

In [ ]:
x.shape

(236115, 20, 2)

In [ ]:
x[20], y[20]

(array([[ 41., 136.],
        [ 39., 137.],
        [ 39., 137.],
        [ 37., 138.],
        [ 37., 138.],
        [ 34., 139.],
        [ 34., 139.],
        [ 32., 140.],
        [ 32., 140.],
        [ 30., 141.],
        [ 30., 141.],
        [ 28., 143.],
        [ 28., 143.],
        [ 25., 144.],
        [ 25., 144.],
        [ 23., 145.],
        [ 23., 145.],
        [ 21., 147.],
        [ 21., 147.],
        [ 18., 148.]]),
 array([ 18., 148.,  15., 149.,  15., 149.,  13., 151.,  13., 151.,  10.,
        152.,  10., 152.,   7., 154.,   7., 154.,   4., 155.]))

In [ ]:
[list(p) for p in predictions[20].numpy().reshape(10,2).astype(int)]

[[19, 149],
 [18, 150],
 [16, 150],
 [15, 151],
 [14, 151],
 [13, 152],
 [12, 152],
 [11, 153],
 [10, 153],
 [9, 154]]

In [ ]:
example = tracks[0:100]

In [ ]:
y[0]

array([ 41., 136.,  39., 137.,  39., 137.,  37., 138.,  37., 138.,  34.,
       139.,  34., 139.,  32., 140.,  32., 140.,  30., 141.])

In [ ]:
y[0].reshape(10,2)

array([[ 41., 136.],
       [ 39., 137.],
       [ 39., 137.],
       [ 37., 138.],
       [ 37., 138.],
       [ 34., 139.],
       [ 34., 139.],
       [ 32., 140.],
       [ 32., 140.],
       [ 30., 141.]])

In [ ]:
example_preds = []
for track in example:
  pred = []
  for i in range(0, maxlen):
    pred.append(track[i])
  for i in range(0, len(track)-maxlen-predlen):
    seq = track[i : i + maxlen]
    inp = np.zeros((1, maxlen, 2))
    for j, s in enumerate(seq):
      inp[0, j, 0] = s[0][0]
      inp[0, j, 1] = s[0][1]
    ps = np.around(model(inp).numpy().reshape(predlen,2)).astype(int)
    psl = [list(p) for p in ps]
    pred.append((track[maxlen+i][0], psl))
  example_preds.append(pred)

In [ ]:
example[0]

In [ ]:
for i, point in enumerate(example_preds[0]):
  print(i, point)

In [ ]:
with open('predicts.json', 'w') as f:
  json.dump(example_preds, f, default=lambda x: str(x))

In [ ]:
files.download('predicts.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>